## Deliverable 2. Create a Customer Travel Destinations Map.

In [35]:
#import the dependencies
import pandas as pd
import gmaps 
import requests
import os

#import the API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [36]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..","Weather_Database","WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Herbertpur,30.4500,77.7333,89.06,57,95,1.52,IN,overcast clouds
1,1,Cape Town,-33.9258,18.4232,55.89,90,75,11.50,ZA,light rain
2,2,Vanimo,-2.6741,141.3028,85.78,64,56,7.92,PG,broken clouds
3,3,Kavali,14.9167,79.9833,88.75,55,100,19.39,IN,overcast clouds
4,4,Ribeira Grande,38.5167,-28.7000,66.27,86,38,13.02,PT,scattered clouds


In [37]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("what is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
what is the maximum temperature you would like for your trip? 80


In [38]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                   (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
7,7,Vaini,-21.2000,-175.2000,75.36,78,75,9.22,TO,broken clouds
19,19,Kununurra,-15.7667,128.7333,75.18,31,14,13.80,AU,few clouds
23,23,Kapaa,22.0752,-159.3190,78.12,79,100,8.01,US,overcast clouds
29,29,Muroto,33.2833,134.1500,78.73,86,100,7.52,JP,overcast clouds
31,31,Manbij,36.5281,37.9549,78.03,31,0,13.67,SY,clear sky
38,38,Atuona,-9.8000,-139.0333,78.24,79,5,16.15,PF,clear sky
45,45,Norman Wells,65.2820,-126.8329,77.04,44,20,18.41,CA,few clouds
53,53,Lianzhou,24.7811,112.3825,77.04,97,100,8.90,CN,overcast clouds
59,59,Shunyi,40.1250,116.6453,75.15,94,40,4.47,CN,scattered clouds
61,61,Marsa Matruh,31.3525,27.2453,77.20,78,0,6.91,EG,clear sky


In [61]:
# 4a. Determine if there are any empty rows.
city_data_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                5
Current Description    0
dtype: int64

In [66]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_city_data_df = city_data_df.dropna(axis="index", how="any")
preferred_city_data_df.count()

City_ID                709
City                   709
Lat                    709
Lng                    709
Max Temp               709
Humidity               709
Cloudiness             709
Wind Speed             709
Country                709
Current Description    709
dtype: int64

In [68]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Herbertpur,IN,89.06,overcast clouds,30.4500,77.7333,
1,Cape Town,ZA,55.89,light rain,-33.9258,18.4232,
2,Vanimo,PG,85.78,broken clouds,-2.6741,141.3028,
3,Kavali,IN,88.75,overcast clouds,14.9167,79.9833,
4,Ribeira Grande,PT,66.27,scattered clouds,38.5167,-28.7000,
5,Acapulco,MX,81.07,broken clouds,16.8634,-99.8901,
6,Arraial Do Cabo,BR,69.96,clear sky,-22.9661,-42.0278,
7,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,
8,Rikitea,PF,74.43,light rain,-23.1203,-134.9692,
9,Mataura,NZ,51.48,scattered clouds,-46.1927,168.8643,


In [74]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... s

In [75]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   709
Country                709
Max Temp               709
Current Description    709
Lat                    709
Lng                    709
Hotel Name             709
dtype: int64

In [80]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [88]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [89]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [92]:
fig.save("WeatherPy_vacation_map.png")

AttributeError: 'Figure' object has no attribute 'save'